# 63. Agregaciones. Funciones count, countDistinct, aprox_count_distinct

In [2]:
%load_ext autoreload
%autoreload 2

import os
path = os.environ.get("HOME")
os.chdir(f"{path}/github/pyspark-hand-on")
from src.utils.utils import spark_session
from pyspark.sql.functions import col, sum, sumDistinct, avg, count
spark = spark_session()

25/10/30 00:45:45 WARN Utils: Your hostname, personal-computer resolves to a loopback address: 127.0.1.1; using 172.17.0.1 instead (on interface docker0)
25/10/30 00:45:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/30 00:45:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.parquet("./data/vuelos.parquet")
df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [6]:
## sum
df.select(sum("DISTANCE").alias("sum_distancia")).show()

+-------------+
|sum_distancia|
+-------------+
|   4785357409|
+-------------+



In [8]:
## sumDistinct (ignora cantidades repetidas y toma un solo valor)
df.select(sumDistinct("DISTANCE").alias("sum_dis_dif")).show()

+-----------+
|sum_dis_dif|
+-----------+
|    1442300|
+-----------+



In [14]:
## avg
df.select(avg("AIR_TIME").alias("promedio_aire"), (sum("AIR_TIME") / count("AIR_TIME")).alias("prom_manual")).show()

+------------------+------------------+
|     promedio_aire|       prom_manual|
+------------------+------------------+
|113.51162809012519|113.51162809012519|
+------------------+------------------+



### Agregación con agrupación

In [7]:
from pyspark.sql.functions import desc

In [8]:
(df.groupBy("ORIGIN_AIRPORT")
    .count()
    .orderBy(desc("count"))
).show()

+--------------+------+
|ORIGIN_AIRPORT| count|
+--------------+------+
|           ATL|346836|
|           ORD|285884|
|           DFW|239551|
|           DEN|196055|
|           LAX|194673|
|           SFO|148008|
|           PHX|146815|
|           IAH|146622|
|           LAS|133181|
|           MSP|112117|
|           MCO|110982|
|           SEA|110899|
|           DTW|108500|
|           BOS|107847|
|           EWR|101772|
|           CLT|100324|
|           LGA| 99605|
|           SLC| 97210|
|           JFK| 93811|
|           BWI| 86079|
+--------------+------+
only showing top 20 rows



In [9]:
(df.groupBy("ORIGIN_AIRPORT", "DESTINATION_AIRPORT")
    .count()
    .orderBy(desc("count"))
).show()

+--------------+-------------------+-----+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|count|
+--------------+-------------------+-----+
|           SFO|                LAX|13744|
|           LAX|                SFO|13457|
|           JFK|                LAX|12016|
|           LAX|                JFK|12015|
|           LAS|                LAX| 9715|
|           LGA|                ORD| 9639|
|           LAX|                LAS| 9594|
|           ORD|                LGA| 9575|
|           SFO|                JFK| 8440|
|           JFK|                SFO| 8437|
|           OGG|                HNL| 8313|
|           HNL|                OGG| 8282|
|           LAX|                ORD| 8256|
|           ATL|                LGA| 8234|
|           LGA|                ATL| 8215|
|           ATL|                MCO| 8202|
|           MCO|                ATL| 8202|
|           SFO|                LAS| 7995|
|           ORD|                LAX| 7941|
|           LAS|                SFO| 7870|
+----------

In [5]:
df.groupBy("MONTH").agg(count("ARRIVAL_DELAY")).show()

+-----+--------------------+
|MONTH|count(ARRIVAL_DELAY)|
+-----+--------------------+
|    1|              457013|
|    6|              492847|
|    3|              492138|
|    5|              489641|
|    9|              462153|
|    4|              479251|
|    8|              503956|
|    7|              514384|
|   10|              482878|
|    2|              407663|
|   12|              469717|
|   11|              462367|
+-----+--------------------+



### Varias agregaciones por grupo:

In [4]:
from pyspark.sql.functions import count, min, max, avg, desc

In [8]:
df.groupBy("ORIGIN_AIRPORT").agg(
    count("AIR_TIME").alias("tiempo_aire"),
    min("AIR_TIME").alias("minimo"),
    max("AIR_TIME").alias("maximo")
).orderBy(desc("tiempo_aire")).show()

+--------------+-----------+------+------+
|ORIGIN_AIRPORT|tiempo_aire|minimo|maximo|
+--------------+-----------+------+------+
|           ATL|     343506|    15|   614|
|           ORD|     276554|    13|   571|
|           DFW|     232647|    11|   534|
|           DEN|     193402|    12|   493|
|           LAX|     192003|    14|   409|
|           PHX|     145552|    19|   444|
|           SFO|     145491|     8|   389|
|           IAH|     144019|    15|   524|
|           LAS|     131937|    25|   429|
|           MSP|     111055|    14|   537|
|           SEA|     110178|    17|   412|
|           MCO|     109532|    25|   395|
|           DTW|     106992|    15|   341|
|           BOS|     104804|    16|   432|
|           CLT|      99052|    17|   379|
|           EWR|      98341|    21|   683|
|           SLC|      96505|    18|   419|
|           LGA|      94834|    19|   311|
|           JFK|      91663|    29|   690|
|           BWI|      84329|    19|   398|
+----------

In [9]:
df.groupBy("MONTH").agg(
    count("ARRIVAL_DELAY").alias("conteo_retraso"),
    avg("DISTANCE").alias("prom_dist")
).orderBy(desc("conteo_retraso")).show()

+-----+--------------+-----------------+
|MONTH|conteo_retraso|        prom_dist|
+-----+--------------+-----------------+
|    7|        514384|841.4772794487611|
|    8|        503956|834.8244276603413|
|    6|        492847|835.6302716626612|
|    3|        492138|816.0553268611494|
|    5|        489641|823.3230588760807|
|   10|        482878|816.4436127652134|
|    4|        479251|817.0060476016745|
|   12|        469717|837.8018926194103|
|   11|        462367|820.2482434846529|
|    9|        462153|815.8487523282274|
|    1|        457013|803.2612794913696|
|    2|        407663| 800.785449834689|
+-----+--------------+-----------------+

